In [2]:
# required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# naviagete to get route name and route
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
time.sleep(3)

scrolling = True
while scrolling:
    old_page = driver.page_source
    body = driver.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(4)
    new_page = driver.page_source
    if new_page == old_page:
        scrolling = False

page1 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route1 = [i.text for i in page1]
link1 = [i.get_attribute('href') for i in page1]
pepsu1 = pd.DataFrame({'route':route1,'route_link':link1})

time.sleep(3)
driver.find_element(By.XPATH,"//div[contains(text(),'2')]").click()
page2 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route2 = [i.text for i in page2]
link2 = [i.get_attribute('href') for i in page2]
pepsu2 = pd.DataFrame({'route':route2,'route_link':link2})

pepsu = pd.concat([pepsu1,pepsu2], ignore_index=True)


pepsu_url = pepsu['route_link']

In [3]:
pepsu

,route,route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
import pandas as pd
import numpy as np

def initialize_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def scroll_to_bottom(driver):
    """Scrolls to the bottom of the page to ensure all content is loaded."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolling = True
    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

def get_bus_data(urls):
    """Fetch bus data for each URL in the list."""
    all_bus_data = pd.DataFrame()
    driver = initialize_driver()

    for url in urls:
        try:
            driver.get(url)
            time.sleep(10)

            # Attempt to click the view buses button
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]'))
                )
                button.click()
            except TimeoutException:
                pass  # Continue if button not found or not clickable

            # Scroll to load all data
            scroll_to_bottom(driver)

            # Extract data
            bus_route = driver.find_element(By.CSS_SELECTOR, 'h1[class="D136_h1"]').text
            bus_name = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')]
            bus_type = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')]
            departure_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')]
            duration = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')]
            arrival_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')]
            star_rating = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')]
            price = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')]
            seats_available = [int(i.text.split()[0]) for i in driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')]

            # Pad lists with NaN to match max length
            max_length = max(len(bus_name), len(bus_type), len(departure_time), len(duration),
                             len(arrival_time), len(star_rating), len(price), len(seats_available))

            def pad_list(lst, length):
                return lst + [np.nan] * (length - len(lst))

            bus_name = pad_list(bus_name, max_length)
            bus_type = pad_list(bus_type, max_length)
            departure_time = pad_list(departure_time, max_length)
            duration = pad_list(duration, max_length)
            arrival_time = pad_list(arrival_time, max_length)
            star_rating = pad_list(star_rating, max_length)
            price = pad_list(price, max_length)
            seats_available = pad_list(seats_available, max_length)

            # Create DataFrame for current URL
            pepsu_bus = pd.DataFrame({
                'Route Name': [bus_route] * max_length,
                'Route Link': [url] * max_length,
                'Bus Name': bus_name,
                'Bus Type': bus_type,
                'Departure Time': departure_time,
                'Duration': duration,
                'Arrival Time': arrival_time,
                'Star Rating': star_rating,
                'Price': price,
                'Seats Available': seats_available
            })

            # Append to main DataFrame
            all_bus_data = pd.concat([all_bus_data, pepsu_bus], ignore_index=True)

        except WebDriverException:
            print(f"Encountered an issue with URL: {url}. Moving to the next.")

    driver.quit()
    return all_bus_data

def process_bus_data(df):
    """Processes bus data by converting columns to numeric and filling NaNs."""
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Star Rating'] = pd.to_numeric(df['Star Rating'], errors='coerce')
    df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors='coerce')

    df['Price'] = df['Price'].fillna(df['Price'].mean().round(0).astype(int))
    df['Star Rating'] = df['Star Rating'].fillna(df['Star Rating'].mean().round(0).astype(int))
    df['Seats Available'] = df['Seats Available'].fillna(df['Seats Available'].mean().round(0).astype(int))
    
    return df

# Replace `bsrtc['route_link']` with the actual list of URLs.
urls = pepsu['route_link']
all_bus_data = get_bus_data(urls)
processed_bus_data = process_bus_data(all_bus_data)
processed_bus_data

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Delhi to Patiala Bus,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67160,Volvo AC Seater (2+2),11:46,05h 29m,17:15,4.4,711.0,35.0
1,Delhi to Patiala Bus,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66665,Volvo AC Seater (2+2),14:01,04h 59m,19:00,4.2,711.0,36.0
2,Delhi to Patiala Bus,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66689,Volvo AC Seater (2+2),15:25,05h 00m,20:25,4.0,711.0,37.0
3,Delhi to Patiala Bus,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67140,AC Seater Hvac 2+2,16:00,05h 30m,21:30,2.5,454.0,50.0
4,Delhi to Patiala Bus,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:31,04h 59m,22:30,4.2,711.0,32.0
...,...,...,...,...,...,...,...,...,...,...
880,Delhi Airport to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-airpor...,Big Bus,A/C Seater / Sleeper (2+2),17:35,06h 05m,23:40,2.0,550.0,32.0
881,Delhi Airport to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-airpor...,RK Holiday,A/C Seater / Sleeper (2+2),18:40,06h 30m,01:10,1.0,1232.0,32.0
882,Delhi Airport to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-airpor...,ADTC tour and Travels,Bharat Benz A/C Seater / Sleeper (2+2),22:30,06h 15m,04:45,4.0,1232.0,32.0
883,Delhi Airport to Phagwara Bus,https://www.redbus.in/bus-tickets/delhi-airpor...,NueGo,Electric A/C Seater (2+2),22:00,08h 10m,06:10,4.0,1232.0,32.0


In [5]:
processed_bus_data.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [6]:
processed_bus_data.to_csv("pepsu.csv", index = False)